## Conditional graph:

Main Goal: `add_conditional_edges()`

**Objectives:**
- Implement conditional logic to routes to different node.
- Use `START` and `END` nodes.
- Create a `router node` to handle decision making.

**Define State, Nodes Function**

In [1]:
from typing import TypedDict

class AgentState(TypedDict):
    num1 : int
    num2 : int
    operation: str
    result: int

def add_nums(state: AgentState) -> AgentState:
    result = state["num1"] + state["num2"]
    state["result"] = result
    return state

def subtract_nums(state: AgentState) -> AgentState:
    result = state["num1"] - state["num2"]
    state["result"] = result
    return state

Now define a **router node:**, note that this doesnt returns a state dict.

In [2]:
def decide_next_node(state: AgentState) -> str:
    """Decision edge: returns an edge to me mapped in add_conditional_edge"""
    if state["operation"] == "+":
        return "add_operation"
    if state["operation"] == "-":
        return "subtract_operation"

**Now structure a node edges.**

In [3]:
from langgraph.graph import StateGraph

from langgraph.graph import START, END

graph = StateGraph(AgentState)

graph.add_node("add", add_nums)
graph.add_node("subtract", subtract_nums)

# Add a router node, Just a RunnablePassThrough
graph.add_node("router", lambda state: state)  # RunnablePassThrough: Does Nothing

**Add edges**

In [4]:
# Start from router
graph.add_edge(START, "router")

# conditional branching edge
graph.add_conditional_edges(
    "router",
    decide_next_node,
    {
        # Edge : Node
        "add_operation": "add",
        "subtract_operation": "subtract"
    }
)

# Define END:
graph.add_edge("add", END)
graph.add_edge("subtract", END)

**Compile, Preview Invoke**

In [7]:
agent = graph.compile()

agent.invoke({
    "num1": 1,
    "num2": 3,
    "operation": "-"
})

{'num1': 1, 'num2': 3, 'operation': '-', 'result': -2}

In [6]:
from IPython.display import display, Markdown

mermaid_code = agent.get_graph().draw_mermaid()
display(Markdown(f"```mermaid\n{mermaid_code}\n```"))


```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	add(add)
	subtract(subtract)
	router(router)
	__end__([<p>__end__</p>]):::last
	__start__ --> router;
	router -. &nbsp;add_operation&nbsp; .-> add;
	router -. &nbsp;subtract_operation&nbsp; .-> subtract;
	add --> __end__;
	subtract --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```